# Hybrid excited rotor

In [ ]:
%matplotlib ipympl
import matplotlib.pyplot as plt
import numpy as np
import sys, os
sys.path.append(os.path.relpath("..//..//..//..//src//python"))
import hiped as hp
from ngsolve import *
from ngsolve.webgui import Draw
from IPython.display import clear_output
from copy import deepcopy

# 1) Geometry

In [ ]:
from geometry import poleMotor
Npp=4
mesh = poleMotor(Npp, maxh = 0.004)
Draw(mesh)

# 2) Material definition
## a) Air

In [ ]:
nullScalar = lambda u : CF(0)
nullVector = lambda b : CF((0,0))
nullMatrix = lambda b : CF(((0,0),(0,0)), dims = (2,2))

mAir = hp.VertexFunction(label = "M=0", f = nullVector , dfdu = nullMatrix, dimInput = 2, dimOutput = 2, flagNGSolve = True)
JAir = hp.VertexFunction(label = "J=0", f = nullScalar , dfdu = nullScalar, dimInput = 2, dimOutput = 2, flagNGSolve = True)

## b) Non linear iron

In [ ]:
#  inspired from Altair Flux model, can also be coupled with temperature : 
# https://help.altair.com/flux/Flux/Help/francais/UserGuide/Francais/topics/
# CourbeDeSaturationAnalytiqueControleDuCoudeFonctionExponentielleDeT.htm

# parameters
mur0, Jsat, a0 = 8000, 2., 0.0003

mu0 = 4e-7*pi
R = CF(((0,1),(-1,0)), dims = (2,2))
trR = CF(((0,-1),(1,0)), dims = (2,2))

def mScalar(normB) : 
    slp = 1-1/mur0
    dab = slp/Jsat
    aa = dab*normB
    bb = aa + 1
    cc =  (1-a0)
    return Jsat/(2*cc) * ( bb -sqrt( bb**2 - 4*aa * cc ) )

def dmscalardb(normB) :
    slp = 1-1/mur0
    dab = slp/Jsat
    aa = dab*normB
    bb = aa + 1
    cc =  (1-a0)
    dab = slp/Jsat
    return Jsat/(2*cc) * dab * ( 1 -  (bb - 2 * cc)/sqrt(bb**2 - 4*aa * cc) ) 

def mVec(b):
    normb = Norm(b)
    return mScalar(normb) * b/normb
        
def dmVec(b):
    normb =  Norm(b)
    bxb = OuterProduct(b, b)
    return dmscalardb(normb)/(normb**2) * bxb + mScalar(normB)/(normb**3) * OuterProduct(tR*b, tR*b)

b = np.arange(0,2,0.01)
h = (b - mScalar(b))/mu0
plt.figure()
plt.plot(h,b)
plt.grid(); plt.xlabel('H (A/m)'); plt.ylabel('B (T)')
plt.show()

In [ ]:
# Magnetic polarization
mbIron = lambda b: mVec(b+CF((1e-12,0))).Compile()
dmbIron = lambda b: dmVec(b+CF((1e-12,0))).Compile()
mIron = hp.VertexFunction(label = "Miron", f = mbIron , dfdu = dmbIron, dimInput = 2, dimOutput = 2, flagNGSolve = True)

# Current density
Jiron = JAir

## b) Electrical DC conductors
- Positive conductor
- Negative conductor

In [ ]:
J_DC = 1e7 # A/m2

# Magnetic polarization
mDC = [mAir]*2

# Current density
JDCp = hp.VertexFunction(label = "+Jdc", f = lambda u : CF(J_DC), dfdu = nullScalar, flagNGSolve = True)
JDCm = hp.VertexFunction(label = "-Jdc", f = lambda u : CF(-J_DC), dfdu = nullScalar, flagNGSolve = True)

## c) Magnets

In [ ]:
Br = 1          # remanent flux density
Nmag = 2*Npp+1  # number of orientations
phi = 0         # common angle shift of all magnets  

# Magnetic polarization
mMag = [hp.VertexFunction(label = "M"+f"{theta*180/pi:.0f}°",
                          f =  lambda b, th= (theta+phi) : CF((cos(th), sin(th))) ,
                          dfdu = nullMatrix, dimInput = 2, dimOutput= 2,
                          flagNGSolve = True) for theta in np.linspace(0,2*pi,Nmag + 1)]
mMag = mMag[:-1]

# Current density
JMag = [JAir] * Nmag

# 3) Interpolation of physical properties
## a) Magnetic polarization
Start from the leaves of the interpolation tree toward the root.

In [ ]:
## DC ##
d2 = hp.Domain(2)
DC_M =  hp.Interpolation(d2, mDC, label = "DC") # no need for penalization here since mDC = 0

## Magnets ##
#######################################################
penal_M_magnets = hp.Penalization("simp",1) # to set !
#######################################################
dN = hp.Domain(Nmag)
mag_M =  hp.Interpolation(dN, mMag, label = "Magnets", penalization= penal_M_magnets)

## Singletons just for nice vizualization
# don't penalize the singleton (the associated ShapeFunction is always 1)
# Penalization will occur directly at the root
d1 = hp.Domain(1)
iron_M = hp.Interpolation(d1, [mIron], label = "Iron")
air_M = hp.Interpolation(d1, [mAir], label = "Air")

## Root ##
###########################################################
penal_M_magnets_root = hp.Penalization("simp",1)    # to set !
penal_M_iron_root = hp.Penalization("simp",1)   # to set !
###########################################################
children_M = [DC_M, mag_M, iron_M, air_M]
penal_M_root = [hp.Penalization("simp",1), penal_M_magnets_root, penal_M_iron_root, hp.Penalization("simp",1)]

d4_3D = hp.Domain("tetra")
interpM =  hp.Interpolation(d4_3D, children_M, label = "Root", penalization= penal_M_magnets)

# Display
plt.figure() ; interpM.plot()
plt.title("M interpolation"); plt.show()

## b) Current density
Keep the **same structure** and **same interpolations labels** as previously, but the penalizations can be different.

In [ ]:
## DC ##
###############################################
penal_J = hp.Penalization("simp",1) # to set !
###############################################
DC_J =  hp.Interpolation(d2, [JDCp, JDCm], label = "DC", penalization= penal_J)

## Magnets ##
mag_J =  hp.Interpolation(dN, JMag, label = "Magnets") # no need for penalization here since JMag = 0

## Singletons just for nice vizualization
iron_J = hp.Interpolation(d1, [Jiron], label = "Iron") 
air_J = hp.Interpolation(d1, [JAir], label = "Air") 

## Root ##
###########################################################
penal_J_root = hp.Penalization("simp",1)        # to set !
###########################################################
children_J = [DC_J, mag_J, iron_J, air_J]
interpJ =  hp.Interpolation(d4_3D, children_J, label = "Root", penalization = penal_J_root)

# Display
plt.figure() ; interpJ.plot()
plt.title("J interpolation"); plt.show()

## c) Definitions of interpolation variables
The interpolation variables are shared by all material properties, which explains why their associated interpolation structure should be the same (also with same labels). The easiest way to define the optimization variable is to use the `setInitialVariable` of one of the interpolation object. Don't forget to project them onto the domain, since some may be located outside !

In [ ]:
fesRho = L2(mesh, definedon = "Rotor") # don't use vector space here, the dimension is automatically set. H1 also works but L2 is recommended
np.random.seed(seed = 0)
rho = interpJ.setInitialVariable(typeInit = "rand", radius = 1, NGSpace = fesRho) # typeInit can be "rand" or "zero"

# display initial variables
plt.figure() ; interpJ.plot(rho)
plt.title("Initial optimization variables"); plt.show()

# after projection
rho = interpJ.projection(rho)
plt.figure() ; interpJ.plot(rho)
plt.title("Initial optimization variables (projected)"); plt.show()

### Specificity

For the following optimization on a hybrid-flux machine rotor we also need the case where the current density is negative. It is equivalent to *swap* the two conductors in the current density interpolation.

In [ ]:
interpJp = interpJ
interpJm = deepcopy(interpJp)
interpJm.Children[0].Children = [JDCm, JDCp]
plt.figure() ; interpJp.plot()
plt.title("Positive current interpolation structure"); plt.show()
plt.figure() ; interpJm.plot()
plt.title("Negative current interpolation structure"); plt.show()

# 4) Solver

In [ ]:
# finite element calculation
from ngsolve.solvers import Newton

fesU = Periodic(H1(mesh, dirichlet = "a0"), [-1,-1,-1])
a_, aStar = fesU.TnT()

def MagnetostaticsWeakForm(rho, w):
    b = R*grad(a_)
    eq = 1/mu0 * grad(aStar) * grad(a_) * dx
    eq += 1/mu0 * grad(aStar) * (R * interpM.eval(rho, b, w)) * dx("Rotor")
    return eq

def solveStateP(rho, w, aInit = GridFunction(fesU)):
    # Definition of the equation
    K = BilinearForm(fesU)
    K += MagnetostaticsWeakForm(rho, w)
    K += - interpJp.eval(rho, 0, w) * aStar * dx("Rotor")
    # Solving
    Newton(K,aInit, freedofs = fesU.FreeDofs(), maxit=100, maxerr=1e-8, dampfactor=0.7, inverse="pardiso", printing = False)
    return aInit

def solveStateM(rho, w, aInit = GridFunction(fesU)):
    # Definition of the equation
    K = BilinearForm(fesU)
    K += MagnetostaticsWeakForm(rho, w)
    K += - interpJm.eval(rho, 0, w) * aStar * dx("Rotor")
    # Solving
    Newton(K,aInit, freedofs = fesU.FreeDofs(), maxit=100, maxerr=1e-10, dampfactor=0.7, inverse="pardiso", printing = False)
    return aInit

w, dwdx = interpM.evalBasisFunction(rho)
aP =  solveStateP(rho, w)
Draw(aP, mesh)

# 5) Objective function and adjoint

In [ ]:
areaAirgap = Integrate(CF(1) * dx("Airgap"), mesh)
perimeterAirgap = Integrate(CF(1) * ds("e1"), mesh)
thicknessAirgap = areaAirgap/perimeterAirgap

ur = Normalize(CF((x,y)))
R = CF(((0,1),(-1,0)), dims = (2,2))

def flux(a):
    return Integrate((R*grad(a))*ur*dx("Airgap"), mesh)/thicknessAirgap

def dflux(aStar):
    return (R*grad(aStar))*ur*dx("Airgap")

def solveAdjoint(a, rho, w):
    p = fesU.TrialFunction()
    rhs = LinearForm(dflux(aStar))
    K = BilinearForm(fesU)
    K += MagnetostaticsWeakForm(rho, w)
    K.AssembleLinearization(a.vec)
    rhs.Assemble()
    gfu = GridFunction(fesU)
    gfu.vec.data = -1* K.mat.Inverse(fesU.FreeDofs(), inverse = "sparsecholesky") * rhs.vec
    return gfu

pP = solveAdjoint(aP,rho, w)
Draw(pP, mesh)

# 6) Gradient of the objective function

In [ ]:
#################################
gamma = 0 # gamma coefficient in [-1,1]
# gamma in {-1,1} leads to Halbach machine
# gamma = 0 leads to wound field machine
# intermediate gamma leads to hybrid flux machine
#################################

def gradRho(u, p, rho, M, J, w, dwdx):
    b = R * grad(u)
    dmdrho = M.evaldx(rho, b, w, dwdx)
    djdrho = J.evaldx(rho, b, w, dwdx)
    g = rho.copy()
    for k in  g.keys():
        g[k] = [1/mu0 * grad(p) * (R * dmdrho[k][i]) - djdrho[k][i] * p for i in range(len(djdrho[k]))]
    return g

def Jobj(ap,am):
    return (gamma + 1)/2 * flux(ap) + (gamma - 1)/2 * flux(am)

def gradJobj(ap, am, pp, pm, rho, w, dwdx):
    gradAp = gradRho(ap, pp, rho, interpM, interpJp, w, dwdx)
    gradAm = gradRho(am, pm, rho, interpM, interpJm, w, dwdx)
    g = rho.copy()
    for k in g.keys():
        g[k] = [ (gamma + 1)/2 * gradAp[k][i] + (gamma - 1)/2 * gradAm[k][i] for i in range(len(gradAp[k]))]
    return g


# 7) Optimization
## a) Initialization

In [ ]:
## Initialization

alpha = 0.1       # Initial step
alpha_min = 1e-4  # Minimal step
n_max = 100      # Maximum number of iterations
n = 0

w, dwdx = interpM.evalBasisFunction(rho)
ap = solveStateP(rho, w)
am = solveStateM(rho, w)
objectiveHistory = [Jobj(ap,am)]
rhoHistory = [deepcopy(rho)]

## b) Optimization loop

In [ ]:
failureLineSearch = False
while( n < n_max and alpha > alpha_min):
    
    if failureLineSearch: # Line search failure : recompute state
        w, dwdx = interpM.evalBasisFunction(rho)

        # 1) State :
        ap = solveStateP(rho, w, aInit = ap)
        am = solveStateM(rho, w, aInit = am)
        
    else : # Line search success : update
        # 2) Adjoint :
        pp = solveAdjoint(ap, rho, w)
        pm = solveAdjoint(am, rho, w)
        
        # 3) Gradient computation :
        gradient = gradJobj(ap, am, pp, pm, rho, w, dwdx)
        normG = dict()
        for key in gradient.keys():
            sq = CF(0)
            for d in range(len(gradient[key])):
                sq = sq + gradient[key][d]**2
            normG[key] = sqrt(sq)
    
    # 4) Update :
    rho_test = deepcopy(rho)
    for key in gradient.keys():
        for d in range(len(gradient[key])):
            rho_test[key][d].Set(rho_test[key][d] - alpha * gradient[key][d]/normG[key]) 
    n += 1
    
    # 5) Projection :
    
    rho_test = interpM.projection(rho_test)
    w, dwdx = interpM.evalBasisFunction(rho_test)
    
    # 6) Step size control :
    ap = solveStateP(rho_test, w, aInit = ap)
    am = solveStateM(rho_test, w, aInit = am)
    objectiveHistory.append(Jobj(ap,am))
    clear_output(wait = True)
    
    print(f'it n°{n} ' + (3-int(np.log10(n)))*' ' + 
          f' |     f = {objectiveHistory[-1]:.5e}     |     step = {alpha:.2e}')
    
    if objectiveHistory[-1] >= objectiveHistory[-2]:
        alpha = alpha/2
        objectiveHistory.pop()
        failureLineSearch = True
    elif objectiveHistory[-1] < objectiveHistory[-2]:
        alpha = alpha*1.2
        rho = rho_test
        rhoHistory.append(deepcopy(rho))
        failureLineSearch = False


# 7) Display of the results
## a) Location of optimization variables

In [ ]:
plt.figure()
interpJ.plot(rhoHistory[-1])
plt.show()

## b) Material class (root)

In [ ]:
w, dwdx = interpJ.evalBasisFunction(rhoHistory[-1])
nMat = 2   # 0 = conductor, 1 = magnets, 2 = iron, 3 = air
Draw(w['Root'][nMat], mesh, min = 0, max = 1)

## c) Conductor type

In [ ]:
nDC = 0   # 0 = positive current, 1 = negative current
Draw(w['Root'][0] * w['DC'][nDC] , mesh, min = 0, max = 1)

## d) Magnet orientation

In [ ]:
nMag = 1   # 0 ... -> NMag
Draw(w['Root'][1] * w['Magnets'][nMag] , mesh, min = 0, max = 1)